In [20]:
import pandas as pd
import numpy as np
import os

from dotenv import load_dotenv
from pymongo import MongoClient
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

First we load the data by connecting to Postgres and Mongo

In [1]:
##connect to postgres db
#engine = create_engine(os.getenv("PG_CONN"))
#df = pd.read_sql("SELECT * FROM jobs", engine)

In [ ]:
#jobs = pd.read_sql("SELECT * FROM job", engine)
#com = pd.read_sql("SELECT * FROM company", engine)
#cat = pd.read_sql("SELECT * FROM category", engine)
#locs = pd.read_sql("SELECT * FROM location", engine)

In [25]:
print(client.list_database_names())

# List all collections in your database
print(db.list_collection_names())

['admin', 'config', 'local']
[]


In [23]:
##connect to mongodb
load_dotenv()
client = MongoClient(
    'mongodb://{}:{}@localhost:27017/jobmarketdb?authSource=admin'.format(
        os.getenv('MONGO_USER'),
        os.getenv('MONGO_PASS')
    )
)
db = client['jobmarketdb']
collection = db['adzunajobs']

cursor = collection.find(
    {},                             # all documents
    {"_id": 0, "job_id": 1, "job_description": 1}   # only these fields
)

#df_mongo = pd.DataFrame(list(cursor))
#df_mongo.head()

""


In [49]:
df_mongo = pd.read_json('adzunajobs.json')
df_mongo = df_mongo[['id', 'description', 'salary_min', 'salary_max']]
df_mongo = df_mongo.rename(columns={"id": "job_id", "description": "job_description"})
df_mongo.head()

,job_id,job_description,salary_min,salary_max
0,5494605665,"Du weißt, was Du kannst – und suchst einen Arb...",18.0,20.0
1,5471852220,DIREKTVERMITTLUNG in Festanstellung (keine Ze...,NaN,NaN
2,5491386603,Bereit für den nächsten Karriereschritt? Wir b...,16.0,18.0
3,5484898587,Du willst Dein Fachwissen in einem neuen Arbei...,70000.0,80000.0
4,5492488590,"Unternehmen Ein international tätiges, produzi...",75000.0,95000.0


In [50]:
##combine data into one df
df = jobs.join(locs, how = 'left', on = 'location_id')
df = df.merge(df_mongo)
df.head()

,job_id,title,contract_type,contract_time,created,adref,redirect_url,salary_is_predicted,company_id,location_id,category_id,display_name,latitude,longitude,country,city,job_description,salary_min,salary_max
0,5494004615,Senior Software Engineer FIND - Solr (Remote -...,NaN,full_time,2025-11-13 15:51:45,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ5NDAwNDYxNSIsI...,https://www.adzuna.de/details/5494004615?utm_m...,0,2,2,1,Deutschland,NaN,NaN,Deutschland,NaN,This position is posted by Jobgether on behalf...,NaN,NaN
1,5489819814,(Senior) Softwareentwickler Java / Java EE (m/...,permanent,full_time,2025-11-10 21:16:57,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4OTgxOTgxNCIsI...,https://www.adzuna.de/details/5489819814?utm_m...,0,3,3,1,"Almoshof, Nürnberg",49.49109,11.06150,Deutschland,Nürnberg,DIREKTVERMITTLUNG in Festanstellung (keine Ze...,NaN,NaN
2,5474059769,Gabelstaplerfahrer (m/w/d),NaN,full_time,2025-11-01 06:05:51,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5474059769?utm_m...,0,12,12,1,"Odelzhausen, Dachau (Kreis)",48.30932,11.19440,Deutschland,Dachau (Kreis),Wir suchen Dich! Denn für unsere Kunden brauch...,18.0,20.0
3,5486087702,Java-Software-Architekt (m/w/d),permanent,full_time,2025-11-07 21:18:24,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4NjA4NzcwMiIsI...,https://www.adzuna.de/details/5486087702?utm_m...,0,3,21,1,"Mitte, Berlin",52.52219,13.40933,Deutschland,Berlin,DIREKTVERMITTLUNG in Festanstellung (keine Ze...,NaN,NaN
4,5486087702,Java-Software-Architekt (m/w/d),permanent,full_time,2025-11-07 21:18:24,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4NjA4NzcwMiIsI...,https://www.adzuna.de/details/5486087702?utm_m...,0,3,21,1,"Mitte, Berlin",52.52219,13.40933,Deutschland,Berlin,DIREKTVERMITTLUNG in Festanstellung (keine Ze...,NaN,NaN


In [45]:
#jobs = pd.read_csv("job.csv")
#jobs.head()

In [51]:
# Check data types
print("Data Types:")
print(df.dtypes)
print("\n" + "="*50 + "\n")

# Check missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Percentage', ascending=False))
print("\n" + "="*50 + "\n")

# Basic statistics
print("Basic Statistics:")
print(df.describe())

Data Types:
job_id                   int64
title                   object
contract_type           object
contract_time           object
created                 object
adref                   object
redirect_url            object
salary_is_predicted      int64
company_id               int64
location_id              int64
category_id              int64
display_name            object
latitude               float64
longitude              float64
country                 object
city                    object
job_description         object
salary_min             float64
salary_max             float64
dtype: object


Missing Values:
               Missing_Count  Percentage
salary_max                 9   69.230769
salary_min                 9   69.230769
contract_type              5   38.461538
city                       2   15.384615
latitude                   2   15.384615
longitude                  2   15.384615
contract_time              1    7.692308


Basic Statistics:
             job_id

**Data Prep and Cleaning**

In [52]:
# Create a copy for cleaning
df_clean = df.copy()

# 1. Handle missing values in numeric columns (latitude, longitude)
print("Handling missing values...")

# df_clean = df_clean.dropna(subset=['latitude', 'longitude'])

# 2. Handle missing values in categorical columns (city, country)
# Fill with 'Unknown' or mode
df_clean['city'].fillna('Unknown', inplace=True)
df_clean['country'].fillna('Deutschland', inplace=True)  

# 3. Handle salary columns - remove rows where salary is missing or invalid
df_clean = df_clean.dropna(subset=['salary_min', 'salary_max'])

# Remove rows where salary_max < salary_min (data quality issue)
df_clean = df_clean[df_clean['salary_max'] >= df_clean['salary_min']]

# Remove extreme outliers in salary
def remove_outliers(df, column, n_std=3):
    mean = df[column].mean()
    std = df[column].std()
    return df[abs(df[column] - mean) <= n_std * std]

df_clean = remove_outliers(df_clean, 'salary_max', n_std=3)

print(f"\nRows after cleaning: {len(df_clean)} (removed {len(df) - len(df_clean)} rows)")
print(f"Missing values remaining:\n{df_clean.isnull().sum()[df_clean.isnull().sum() > 0]}")

Handling missing values...

Rows after cleaning: 4 (removed 9 rows)
Missing values remaining:
contract_type    4
dtype: int64


/tmp/ipykernel_5176/1385761668.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['city'].fillna('Unknown', inplace=True)
/tmp/ipykernel_5176/1385761668.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

**Keyword extraction from job description**

In [ ]:
# Fill missing descriptions
df_clean['job_description'] = df_clean['job_description'].fillna('')

# Convert to lowercase for consistency
df_clean['desc_lower'] = df_clean['job_description'].str.lower()

# Define comprehensive keyword categories
keywords_dict = {
    # Seniority levels
    'seniority': ['senior', 'lead', 'principal', 'staff', 'junior', 'entry', 
                  'experienced', 'expert', 'chief', 'head of', 'director', 'vp'],
    
    # Technical skills - Programming languages
    'languages': ['python', 'java', 'javascript', 'typescript', 'c++', 'c#', 
                  'ruby', 'go', 'rust', 'scala', 'kotlin', 'swift', 'php', 'r'],
    
    # Frameworks & Technologies
    'frameworks': ['react', 'angular', 'vue', 'django', 'flask', 'spring', 
                   'node.js', 'express', '.net', 'tensorflow', 'pytorch', 'keras'],
    
    # Databases
    'databases': ['sql', 'mysql', 'postgresql', 'mongodb', 'oracle', 'redis', 
                  'elasticsearch', 'cassandra', 'dynamodb', 'neo4j'],
    
    # Cloud & DevOps
    'cloud': ['aws', 'azure', 'gcp', 'cloud', 'docker', 'kubernetes', 'k8s',
              'terraform', 'ansible', 'jenkins', 'gitlab', 'ci/cd', 'devops'],
    
    # Data & AI
    'data_ai': ['machine learning', 'deep learning', 'ai', 'data science', 
                'analytics', 'big data', 'etl', 'data warehouse', 'spark', 
                'hadoop', 'tableau', 'power bi'],
    
    # Methodologies
    'methodologies': ['agile', 'scrum', 'kanban', 'waterfall', 'test-driven',
                      'microservices', 'rest', 'api', 'graphql'],
    
    # Soft skills & Management
    'management': ['team lead', 'leadership', 'management', 'communication',
                   'project management', 'stakeholder', 'mentor', 'architect'],
    
    # Education requirements
    'education': ['bachelor', 'master', 'phd', 'doctorate', 'degree', 
                  'university', 'certification', 'certified'],
    
    # Experience indicators
    'experience': ['years of experience', 'year experience', 'experience in',
                   'proven track record', 'extensive experience'],
    
    # Domain expertise
    'domains': ['finance', 'fintech', 'healthcare', 'e-commerce', 'automotive',
                'telecommunications', 'gaming', 'security', 'blockchain', 'iot'],
    
    # Company size indicators
    'company_type': ['startup', 'enterprise', 'corporation', 'sme', 'scale-up',
                     'fortune 500', 'multinational'],
    
    # Benefits (often correlate with higher salaries)
    'benefits': ['remote', 'flexible', 'home office', 'relocation', 'visa',
                 'bonus', 'stock options', 'equity', '30 days vacation']
}

# Count keyword occurrences in each category
for category, keywords in keywords_dict.items():
    df_clean[f'{category}_count'] = 0
    for keyword in keywords:
        df_clean[f'{category}_count'] += df_clean['desc_lower'].str.contains(
            keyword, regex=False, na=False
        ).astype(int)
    print(f"{category}: mean count = {df_clean[f'{category}_count'].mean():.2f}")

# Create binary features for high-impact individual keywords
high_impact_keywords = [
    'senior', 'lead', 'principal', 'architect', 'expert',
    'python', 'java', 'aws', 'azure', 'kubernetes',
    'machine learning', 'ai', 'blockchain', 
    'remote', 'phd', 'team lead'
]

for keyword in high_impact_keywords:
    col_name = f"has_{keyword.replace(' ', '_')}"
    df_clean[col_name] = df_clean['desc_lower'].str.contains(
        keyword, regex=False, na=False
    ).astype(int)

print(f"\nCreated {len(keywords_dict)} category count features")
print(f"Created {len(high_impact_keywords)} binary keyword features")

In [ ]:
# Additional text-based features
df_clean['desc_length'] = df_clean['job_description'].str.len()
df_clean['desc_word_count'] = df_clean['job_description'].str.split().str.len()
df_clean['desc_sentence_count'] = df_clean['job_description'].str.count(r'[.!?]+')

# Extract years of experience mentioned (e.g., "5+ years", "3-5 years")
import re

def extract_years_experience(text):
    if pd.isna(text) or text == '':
        return 0
    # Look for patterns like "5+ years", "3-5 years", "5 years"
    patterns = [
        r'(\d+)\+?\s*(?:years?|jahr)',  # 5+ years, 5 years
        r'(\d+)\s*(?:-|to|bis)\s*\d+\s*(?:years?|jahr)',  # 3-5 years
    ]
    years = []
    for pattern in patterns:
        matches = re.findall(pattern, text.lower())
        years.extend([int(m) for m in matches if m])
    return max(years) if years else 0

df_clean['years_experience_required'] = df_clean['job_description'].apply(extract_years_experience)

# Count technical requirements
df_clean['requirements_count'] = df_clean['desc_lower'].str.count(
    r'\b(require|must|need|essential|mandatory)\b'
)

# Count "nice to have" vs "must have"
df_clean['nice_to_have_count'] = df_clean['desc_lower'].str.count(
    r'\b(nice to have|preferred|bonus|plus|advantage)\b'
)

print("Advanced text features:")
print(df_clean[['desc_length', 'desc_word_count', 'years_experience_required', 
                'requirements_count', 'nice_to_have_count']].describe())

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF features from job descriptions
# Use a limited number of features to avoid overfitting
tfidf = TfidfVectorizer(
    max_features=50,  # Top 50 most important terms
    min_df=5,  # Term must appear in at least 5 documents
    max_df=0.8,  # Ignore terms that appear in more than 80% of documents
    ngram_range=(1, 2),  # Use single words and bi-grams
    stop_words=['german']  # Remove common German words
)

# Fit and transform
tfidf_features = tfidf.fit_transform(df_clean['job_description'].fillna(''))
tfidf_df = pd.DataFrame(
    tfidf_features.toarray(),
    columns=[f'tfidf_{i}' for i in range(tfidf_features.shape[1])],
    index=df_clean.index
)

# Show most important terms
feature_names = tfidf.get_feature_names_out()
print("Top TF-IDF terms:")
print(feature_names[:20])

# Merge TF-IDF features with main dataframe
df_clean = pd.concat([df_clean, tfidf_df], axis=1)

print(f"\nAdded {tfidf_features.shape[1]} TF-IDF features")

# Optional: Show which terms are most correlated with salary
if 'salary_avg' in df_clean.columns:
    correlations = []
    for col in tfidf_df.columns:
        corr = df_clean[col].corr(df_clean['salary_avg'])
        if not pd.isna(corr):
            correlations.append((col, corr, feature_names[int(col.split('_')[1])]))
    
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    print("\nTop 10 salary-correlated terms:")
    for col, corr, term in correlations[:10]:
        print(f"  {term}: {corr:.3f}")

Top TF-IDF terms:
['00' '00 je' 'als' 'arbeitssicherheit' 'arbeitssicherheit in'
 'arbeitsvertrag' 'bei' 'bei diesem' 'bei persona' 'dein' 'der' 'die'
 'du dich' 'erwarten' 'fachkraft arbeitssicherheit' 'fachwissen'
 'freuen kannst' 'für' 'gehalt' 'gutes']

Added 50 TF-IDF features


**Feature engineering**

In [56]:
# Create target variable: average salary
df_clean['salary_avg'] = (df_clean['salary_min'] + df_clean['salary_max']) / 2
df_clean['salary_range'] = df_clean['salary_max'] - df_clean['salary_min']

# Create binary features from categorical variables
df_clean['has_location'] = df_clean['city'].notna().astype(int)

# Extract useful features from display_name (job title)
df_clean['title_length'] = df_clean['display_name'].str.len()
df_clean['title_word_count'] = df_clean['display_name'].str.split().str.len()

# Check for keywords in job titles
title_keywords = ['senior', 'lead', 'manager', 'engineer', 'developer']
for keyword in title_keywords:
    df_clean[f'title_is_{keyword}'] = df_clean['display_name'].str.lower().str.contains(
        keyword, na=False
    ).astype(int)

print("Feature engineering complete!")
print(f"Total features: {len(df_clean.columns)}")

# Show correlation of text features with salary
text_feature_cols = [col for col in df_clean.columns if any(
    x in col for x in ['_count', 'has_', 'desc_', 'years_experience', 'tfidf_']
)]

if 'salary_avg' in df_clean.columns and text_feature_cols:
    correlations = df_clean[text_feature_cols + ['salary_avg']].corr()['salary_avg'].sort_values(ascending=False)
    print("\nTop 15 text features correlated with salary:")
    print(correlations[1:16])  # Skip salary_avg itself

Feature engineering complete!
Total features: 79

Top 15 text features correlated with salary:
tfidf_3     1.000000
tfidf_14    1.000000
tfidf_4     1.000000
tfidf_29    1.000000
tfidf_37    1.000000
tfidf_22    1.000000
tfidf_44    1.000000
tfidf_9     0.864738
tfidf_2     0.730579
tfidf_23    0.548340
tfidf_15    0.337213
tfidf_21    0.337213
tfidf_47    0.337213
tfidf_12    0.185453
tfidf_31   -0.333347
Name: salary_avg, dtype: float64


**Encode categorical variables**

In [37]:
# Select features for modeling
categorical_cols = ['country', 'city']

# Get all text-based features we created
text_feature_cols = [col for col in df_clean.columns if any(
    x in col for x in ['_count', 'has_', 'desc_length', 'desc_word_count', 
                       'years_experience', 'requirements_count', 'nice_to_have_count',
                       'title_is_', 'tfidf_']
)]

numeric_cols = ['latitude', 'longitude', 'title_length', 'title_word_count'] + text_feature_cols

# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    if col in df_clean.columns:
        le = LabelEncoder()
        df_clean[f'{col}_encoded'] = le.fit_transform(df_clean[col].astype(str))
        label_encoders[col] = le

# Frequency encoding for city (better for high cardinality)
city_freq = df_clean['city'].value_counts(normalize=True)
df_clean['city_frequency'] = df_clean['city'].map(city_freq)

print(f"Total numeric/text features: {len(text_feature_cols)}")
print("Encoded features complete")

,created,adref,redirect_url,salary_is_predicted,category_id,job_age_days,title_(Senior) Softwareentwickler Java / Java EE (m/w/d) ab jetzt,title_.NET Software Engineer,title_Bauingenieur Gebäudemanagement (m/w/d),title_CNC-Dreher (m/w/d),...,location_id_40,location_id_41,location_id_42,location_id_44,location_id_45,location_id_46,location_id_47,location_id_48,location_id_49,location_id_50
job_id,,,,,,,,,,,,,,,,,,,,,
5485243050,2025-11-07 13:05:18,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5485243050?utm_m...,0,1,11,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5494004615,2025-11-13 15:51:45,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ5NDAwNDYxNSIsI...,https://www.adzuna.de/details/5494004615?utm_m...,0,1,5,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5489819814,2025-11-10 21:16:57,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4OTgxOTgxNCIsI...,https://www.adzuna.de/details/5489819814?utm_m...,0,1,7,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5482662909,2025-11-05 20:35:43,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5482662909?utm_m...,0,1,12,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5467966903,2025-10-28 14:05:56,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5467966903?utm_m...,0,1,21,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Prepare for data modeling**

In [39]:
# Define feature columns
feature_cols = (numeric_cols + 
                [f'{col}_encoded' for col in categorical_cols if col in df_clean.columns] + 
                ['city_frequency'])

# Remove any features that might leak information or cause issues
feature_cols = [col for col in feature_cols if col in df_clean.columns 
                and col not in ['salary_range', 'salary_avg', 'salary_min', 'salary_max']]

# Remove any columns with all NaN or constant values
valid_features = []
for col in feature_cols:
    if df_clean[col].notna().sum() > 0 and df_clean[col].nunique() > 1:
        valid_features.append(col)
    else:
        print(f"Removing {col}: insufficient variation")

feature_cols = valid_features

X = df_clean[feature_cols]
y = df_clean['salary_avg']

# Handle any remaining NaN values in features
X = X.fillna(0)

print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"\nTotal features: {len(feature_cols)}")
print(f"Text-based features: {len([c for c in feature_cols if any(x in c for x in ['has_', '_count', 'tfidf_', 'years_experience'])])}")
print(f"\nFeature statistics:")
print(X.describe())

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Int64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'nu

**Split data**

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeatures scaled successfully")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb  # Install with: pip install xgboost

# Initialize models
models = {
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(
        n_estimators=200, 
        max_depth=20,
        min_samples_split=5,
        random_state=42, 
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=200,
        max_depth=7,
        learning_rate=0.1,
        random_state=42
    )
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled data for linear models, original for tree-based
    if 'Regression' in name and 'Forest' not in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    }
    
    print(f"{name} Results:")
    print(f"  MAE: €{mae:,.2f}")
    print(f"  RMSE: €{rmse:,.2f}")
    print(f"  R² Score: {r2:.4f}")

In [ ]:
# Get best model (highest R2)
best_model_name = comparison_df.loc[comparison_df['R2'].idxmax(), 'Model']
best_model = results[best_model_name]['model']

print(f"Best Model: {best_model_name}")

# Feature importance for tree-based models
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 20 Most Important Features:")
    print(importance_df.head(20).to_string(index=False))
    
    # Categorize features
    text_features = importance_df[importance_df['Feature'].str.contains('has_|_count|tfidf_|years_experience')]
    location_features = importance_df[importance_df['Feature'].str.contains('city|country|latitude|longitude')]
    
    print(f"\nTop text-based features:")
    print(text_features.head(10).to_string(index=False))
    
    # Visualize top features
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(20)
    plt.barh(top_features['Feature'], top_features['Importance'])
    plt.xlabel('Importance')
    plt.title(f'Top 20 Feature Importances - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # Show keyword category importance
    category_importance = {}
    for category in keywords_dict.keys():
        cat_features = importance_df[importance_df['Feature'].str.contains(category)]
        if len(cat_features) > 0:
            category_importance[category] = cat_features['Importance'].sum()
    
    if category_importance:
        cat_imp_df = pd.DataFrame(list(category_importance.items()), 
                                   columns=['Category', 'Total_Importance']).sort_values('Total_Importance', ascending=False)
        print("\nKeyword category importance:")
        print(cat_imp_df.to_string(index=False))
        
        plt.figure(figsize=(10, 6))
        plt.barh(cat_imp_df['Category'], cat_imp_df['Total_Importance'])
        plt.xlabel('Total Importance')
        plt.title('Keyword Category Importance')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

# For linear models, show coefficients
elif hasattr(best_model, 'coef_'):
    coef_df = pd.DataFrame({
        'Feature': feature_cols,
        'Coefficient': best_model.coef_
    }).sort_values('Coefficient', key=abs, ascending=False)
    
    print("\nTop 20 Most Important Features (by coefficient magnitude):")
    print(coef_df.head(20).to_string(index=False))

**Visualization**

In [ ]:
# Visualize predictions vs actual
best_predictions = results[best_model_name]['predictions']

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot
axes[0].scatter(y_test, best_predictions, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Salary (€)')
axes[0].set_ylabel('Predicted Salary (€)')
axes[0].set_title(f'Actual vs Predicted Salaries - {best_model_name}')

# Residual plot
residuals = y_test - best_predictions
axes[1].scatter(best_predictions, residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted Salary (€)')
axes[1].set_ylabel('Residuals (€)')
axes[1].set_title('Residual Plot')

plt.tight_layout()
plt.show()

# Distribution of errors
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=50, edgecolor='black')
plt.xlabel('Prediction Error (€)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.axvline(x=0, color='r', linestyle='--', label='Perfect Prediction')
plt.legend()
plt.show()

print(f"\nError Statistics:")
print(f"Mean Error: €{residuals.mean():,.2f}")
print(f"Median Error: €{residuals.median():,.2f}")
print(f"Std Error: €{residuals.std():,.2f}")

**Predictions on new data**

In [ ]:
# Function to predict salary for new job listings
def predict_salary(new_data, model, scaler, feature_cols, is_linear=False):
    """
    Predict salary for new job listings
    
    Parameters:
    new_data: DataFrame with same features as training data
    model: trained model
    scaler: fitted StandardScaler
    feature_cols: list of feature column names
    is_linear: whether the model is a linear model (needs scaling)
    """
    # Prepare features
    X_new = new_data[feature_cols]
    
    # Scale if using linear model
    if is_linear:
        X_new_scaled = scaler.transform(X_new)
        predictions = model.predict(X_new_scaled)
    else:
        predictions = model.predict(X_new)
    
    return predictions

# Example: Predict for jobs with different keyword combinations
is_linear = 'Regression' in best_model_name and 'Forest' not in best_model_name

sample_data = X_test.head(10)
sample_predictions = predict_salary(sample_data, best_model, scaler, feature_cols, is_linear)

print("Sample Predictions:")
results_list = []
for i, (idx, row) in enumerate(sample_data.iterrows()):
    actual = y_test.loc[idx]
    predicted = sample_predictions[i]
    error = abs(predicted - actual)
    error_pct = (error / actual) * 100
    
    # Get original job info
    job_title = df_clean.loc[idx, 'display_name']
    city = df_clean.loc[idx, 'city']
    
    results_list.append({
        'Job': job_title[:50],
        'City': city,
        'Predicted': f"€{predicted:,.0f}",
        'Actual': f"€{actual:,.0f}",
        'Error': f"€{error:,.0f}",
        'Error %': f"{error_pct:.1f}%"
    })

results_df = pd.DataFrame(results_list)
print(results_df.to_string(index=False))

**Save Model**

In [ ]:
import pickle

# Save the best model, scaler, and all necessary objects
model_package = {
    'model': best_model,
    'scaler': scaler,
    'feature_cols': feature_cols,
    'label_encoders': label_encoders,
    'tfidf_vectorizer': tfidf,
    'keywords_dict': keywords_dict,
    'high_impact_keywords': high_impact_keywords,
    'model_name': best_model_name,
    'is_linear': is_linear
}

with open('salary_prediction_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print("Model package saved successfully!")
print(f"\nPackage contents:")
for key in model_package.keys():
    print(f"  - {key}")

# To load later:
# with open('salary_prediction_model.pkl', 'rb') as f:
#     saved = pickle.load(f)
#     model = saved['model']
#     scaler = saved['scaler']
#     feature_cols = saved['feature_cols']